In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [44]:
reviews_sample= pd.read_csv(r'C:\Users\shanm\OneDrive\Desktop\Git_data\Sephora_sample\X_reviews_sample.csv',low_memory =True)
reviews_sample.head()

,Unnamed: 0,review_text,label,rating,text_word_count,clean_text,tokenized_text,tokenized_text_2
0,0,Gel type moisturizers aren’t typically my go-t...,1.0,5,51,gel type moisturizers aren’t typically my goto...,"['gel', 'type', 'moistur', '’', 'typic', 'goto...",gel type moistur ’ typic goto one impress gel ...
1,1,smells great! works really well on my acne sca...,1.0,4,44,smells great works really well on my acne scar...,"['smell', 'great', 'work', 'realli', 'well', '...",smell great work realli well acn scare need co...
2,2,Caused me so many breakouts. I thought this wo...,0.0,1,29,caused me so many breakouts i thought this wou...,"['caus', 'mani', 'breakout', 'thought', 'would...",caus mani breakout thought would good use clea...
3,3,I could not be happier with this mask! I alway...,1.0,5,45,i could not be happier with this mask i always...,"['could', 'happier', 'mask', 'alway', 'use', '...",could happier mask alway use overnight mask am...
4,4,Fastest acting spot reducer I have ever tried....,1.0,5,50,fastest acting spot reducer i have ever tried ...,"['fastest', 'act', 'spot', 'reduc', 'ever', 't...",fastest act spot reduc ever tri got watch one ...


In [45]:
reviews_sample.loc[reviews_sample['text_word_count']==409]

,Unnamed: 0,review_text,label,rating,text_word_count,clean_text,tokenized_text,tokenized_text_2
21278,21278,Shiseido has a great reputation with putting o...,1.0,5,409,shiseido has a great reputation with putting o...,"['shiseido', 'great', 'reput', 'put', 'qualiti...",shiseido great reput put qualiti product glad ...


In [46]:
reviews_sample['text_word_count'].describe()

count    156820.000000
mean         58.845606
std          41.760496
min           1.000000
25%          31.000000
50%          49.000000
75%          75.000000
max         409.000000
Name: text_word_count, dtype: float64

In [47]:
X= reviews_sample['clean_text']
y = reviews_sample['label'] 

In [48]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80)

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, TextVectorization
from tensorflow.keras.optimizers import Adam

In [50]:
max_features = 5000  # Maximum vocab size.
max_len = 200  # Sequence length to pad the outputs to.

# Create the layer.
vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

# Now that the vocab layer has been created, call `adapt` on the
# text-only dataset to create the vocabulary. You don't have to batch,
# but for large datasets this means we're not keeping spare copies of
# the dataset.
vectorize_layer.adapt(X_train)

In [51]:
x_train = vectorize_layer(X_train)

In [52]:
x_train[0]

<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([  36,  186,   13,    3,   65,  171,   11,    6,  299,  143,  555,
         16,  151,    3,   11,    5, 1951,  106,   43,   16,   94,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [53]:
x_test = vectorize_layer(X_test)

In [54]:
x_train=np.array(x_train)

In [55]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # encoding
y_train_label= le.fit_transform(y_train)
y_test_label= le.transform(y_test)

In [56]:
from collections import Counter

Counter(y_train_label).keys() # equals to list(set(words))
Counter(y_train_label).values() # counts the elements' frequency

dict_values([15683, 15681])

In [57]:
x_train.shape,y_train_label.shape

((31364, 200), (31364,))

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(max_features,64,input_length = 200))
model.add(Bidirectional(LSTM(64)))
# model.add(Dropout(0.2))
# model.add(LSTM(64))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 200, 64)           160000    
                                                                 
 bidirectional_7 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 25)                3225      
                                                                 
 dense_11 (Dense)            (None, 1)                 26        
                                                                 
Total params: 229,299
Trainable params: 229,299
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(x_train,y_train_label, epochs = 2)

Epoch 1/2
981/981 [==============================] - 71s 70ms/step - loss: 0.3503 - accuracy: 0.8495
Epoch 2/2
981/981 [==============================] - 66s 67ms/step - loss: 0.2538 - accuracy: 0.90370s - loss: 0.2541 - accu


In [60]:
model.predict(x_test)

array([[0.01733035],
       [0.01139319],
       [0.33395857],
       ...,
       [0.0649789 ],
       [0.07474905],
       [0.9682166 ]], dtype=float32)

In [71]:
model.predict(vectorize_layer(['Gel type moisturizers are not typically my thing']))

array([[0.14532462]], dtype=float32)

In [14]:
y_train_label

array([0, 1, 1, ..., 0, 0, 1], dtype=int64)